In [14]:
import sys

from pathlib import Path
import os
p = Path(os.path.abspath('')).resolve().parents[0]
sys.path.insert(1, str(p))

from idxutils import gen_in_network_links
from mrfutils import InvalidMRF, JSONOpen
from tqdm import tqdm
import gzip
import csv
import hashlib

import logging
logging.basicConfig()
log = logging.getLogger('mrfutils')
log.setLevel(logging.WARNING)



'''
Takes a file with a list of index urls
'''

'\nTakes a file with a list of index urls\n'

In [2]:
input_file = "UHC_indexes.csv"
output_file = "UHC_payers_deduped.csv"

In [3]:
def blocks(files, size=65536):
    while True:
        b = files.read(size)
        if not b: break
        yield b

In [16]:
unique_lines = set()
with open(input_file, "r") as f:
    total = sum(bl.count("\n") for bl in blocks(f))
    f.seek(0)
    with open(output_file, "a", newline="") as output_csv:
        writer = csv.DictWriter(output_csv, fieldnames=["url"])
        # writer.writeheader()
        for filename in tqdm(f, total=total):
            # Index files, needs to a url
            filename = filename.strip("\n").strip('"')
            # filename = JSONOpen(filename)
            try:
                for url in gen_in_network_links(filename):
                    line_hash = hashlib.md5(url.encode()).hexdigest()
                    if line_hash not in unique_lines:
                        unique_lines.add(line_hash)
                        writer.writerow({
                            "url": url
                        })
                    
            except KeyboardInterrupt:
                break
            except:
                pass

39553it [2:48:32,  3.91it/s]                                 
